In [19]:
import os
import pathlib
import restart

# need this ugly variable for now to directly access config
PATH = pathlib.Path(restart.__file__).parent.absolute()

In [20]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

restart = RestartModel(population='oes', state='Alaska')
model = restart.model
chart_colors = ["#77AADD", "#99DDFF", "#44BB99", "#BBCC33", "#AAAA00", "#EEDD88",
                "#EE8866", "#FFAABB", "#DDDDDD", "#000000"]

def generate_pie_chart(df, title="", show_decimal=False):
    fig=plt.figure(title=title)
    
    pie_chart = plt.pie(sizes=df.values.tolist(),
                        labels=df.index.values.tolist(),
                        display_labels="outside",
                        colors=chart_colors[:df.index.values.size],
                        display_values=True)
    if not show_decimal:
        pie_chart.values_format = "0"
    return fig

def generate_bar(df, title="", scientific_notation=False, small_xlabel=False):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    bar_chart = plt.bar(x=x_titles,
                        y=df,
                        colors=chart_colors[:df.index.values.size])
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_group_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="grouped",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_scatter(df, title="", scientific_notation=False, small_xlabel=True):
    fig = plt.figure(title=title)
    x_vals = df.index.values.tolist()
    if len(x_vals) > 5:
        small_xlabel=True
    x_titles = []
    for val in x_vals:
        if len(val.split(' ')) < 3:
            x_titles.append(val)
        else:
            x_titles.append(" ".join(val.split(' ')[:2]))
    scatter = plt.scatter(x=x_titles, y=df)
    
    if small_xlabel:
        fig.axes[0].tick_style = {"font-size": "6"}
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig
    
def generate_stacked_bar(df, title="", scientific_notation=False):
    fig = plt.figure(title=title)
    
    bar_chart = plt.bar(x=df.columns.values.tolist(),
                        y=df,
                        labels=df.index.values.tolist(),
                        display_legend=False,
                        type="stacked",
                        colors=chart_colors[:df.index.values.size])
    if df.columns.name:
        plt.xlabel(df.columns.name.rsplit(" ", 1)[0])
    plt.ylim(0, np.amax(df.values))
    if not scientific_notation:
        fig.axes[1].tick_format = ".1f"
    return fig

def generate_separate_bar_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    bar_list = []
    for col in df.columns: # .values.tolist()
        bar_list.append(generate_bar(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return bar_list

def generate_separate_scatter_list(df, scientific_notation=False, small_xlabel=False): # returns list, NOT widget
    scatter_list = []
    for col in df.columns: # .values.tolist()
        scatter_list.append(generate_scatter(df[col][df[col] != 0], title=col, scientific_notation=scientific_notation, small_xlabel=small_xlabel))
    return scatter_list

def generate_html_legend(df, colors=chart_colors, table=True, font_size=13):
    name = df.index.name.rsplit(" ", 1)[0]
    html_string = f"<div style='font-size:{font_size}px; font-family:helvetica'><b style='font-weight:bold'>{name}</b>"
    indices = df.index.values
    if table:
        html_string += "<table><tr>"
        for i in range(0, indices.size):
            if i % 2 == 0:
                index_num = int(i / 2)
            else:
                index_num = int(i / 2 + indices.size / 2)
            html_string += f"<td style='padding:0 5px'><span style='color:{colors[index_num]}'>█</span> {indices[index_num]}</td>"
            if i % 2 != 0:
                html_string += "</tr><tr>"
        html_string += "</tr></table>"
    else:
        for_count = 0
        for string in indices:
            if for_count == 0:
                html_string += "<br>"
            else:
                html_string += "&emsp;"
            html_string += f"<span style='color:{colors[for_count]}'>█</span> {string}"
            for_count += 1
    html_string += "</div>"
    return widgets.HTML(html_string)

def generate_group_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_group_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

def generate_stacked_bar_legend(df, title="", scientific_notation=False, legend_table=True):
    chart = generate_stacked_bar(df, title=title, scientific_notation=scientific_notation)
    legend = generate_html_legend(df, table=legend_table)
    return widgets.VBox([chart, legend])

burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
burn_chart = generate_group_bar_legend(model.demand.demand_per_unit_map_dn_um.df)

In [21]:
import sys
import pandas as pd
import numpy as np
from restart import RestartModel
from restart import Data
from restart.util import set_config, to_df, to_sheet, display_population, format_population, format_cells
import ipywidgets as widgets
import bqplot
from bqplot import pyplot as plt

model.inventory.set_average_orders_per_period(model.demand.demand_by_popsum1_total_rp1n_tc)

slider = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)

def dashboard(backstop):
    set_stock(backstop)
    
def display_stock(df):
    df_round = df.round()
    index_name = "Population"
    headers = ['Essential', 'Non-Essential']
    df_round.insert(loc=0, column=index_name, value=headers)
    sheet = to_sheet(df_round)
    format_cells(sheet)
    sheet.row_headers = False
    df_chart = df
    df_chart.index = df_chart.index.get_level_values(1)
    chart = generate_group_bar_legend(df_chart, scientific_notation=True)
    display(sheet)
    display(chart)
    
def set_stock(backstop):
    backstop = [backstop]
    model.inventory.order(model.inventory.inv_by_popsum1_total_rp1n_tc)
    model.inventory.set_min_in_periods(backstop)
    display_stock(model.inventory.inv_by_popsum1_total_rp1n_tc.df)
    
wa_burn_sheet = format_population(to_sheet(model.demand.demand_per_unit_map_dn_um.df))
pop = format_population(to_sheet(model.population.population_pP_tr.df))
    
out = widgets.interactive_output(dashboard, {'backstop': slider})

In [26]:
restart_health = RestartModel(population='oes', state='Alaska', subpop='healthcare')
model_health = restart_health.model
pop_health = format_population(to_sheet(model_health.population.population_pP_tr.df))

slider_health = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)
out_health = widgets.interactive_output(dashboard, {'backstop': slider_health})

In [23]:
restart_fr = RestartModel(population='oes', state='Alaska', subpop='wa_tier2_opt2')
model_fr = restart_fr.model
pop_fr = format_population(to_sheet(model_fr.population.population_pP_tr.df))

slider_fr = widgets.IntSlider(min=1, max=120, value=30, description = "Days", continuous_update=False)
out_fr = widgets.interactive_output(dashboard, {'backstop': slider_fr})

In [27]:
import ipyvuetify as v
v.theme.themes.light.primary = 'colors.teal'

v.Tabs(_metadata={'mount_id': 'content-main'}, children=[
    v.Tab(children=['PPE Analysis']),
    v.Tab(children=['General Population']),
    v.Tab(children=['Healthcare Population']),
    v.Tab(children=['Outpatient/First Responders']),
    v.Tab(children=['Burn Rates']),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Covid Incremental Ventilator Requirements"]),
                v.CardText(children=[
                    "Analysis for general population"
                ]),
                slider, out,
                v.CardText(children=[
                    "Analysis for healthcare workers"
                ]),
                slider_health, out_health,
                v.CardText(children=[
                    "Analysis for outpatient healthcare workers and first responders"
                ]),
                slider_fr, out_fr
            ]),
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["BLS P"]),
                v.CardText(children=[
                    "All OES codes in the state of Alaska"
                ]),
                pop
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["BLS Population with High Granularity"]),
                v.CardText(children=[
                    "OES codes filtered for healthcare workers"
                ]),
                pop_health
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, xl4=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["BLS Population with High Granularity"]),
                v.CardText(children=[
                    "OES codes filtered for outpatient healthcare providers and first responders"
                ]),
                pop_fr,
            ])
                
        ])
    ]),
    v.TabItem(children=[
        v.Layout(column=True, wrap=True, align_left=True, children=[
            v.Card(xs12=True, lg6=True, x14=True, children=[
                v.CardTitle(primary_title=True, class_='headline', children=["Burn Rates"]),
                v.CardSubtitle(children=["Per Capita Resource Demand Assumptions"]),
                burn_sheet,
                burn_chart,
            ])
        ])
    ])
])

Tabs(children=[Tab(children=['PPE Analysis']), Tab(children=['General Population']), Tab(children=['Healthcare…